In [1]:
import util
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
#constants
PERCENT_OF_DATA = 0.0125
NUM_ENSEMBLE_MODELS = 4

num_classes = 2
model_save_path = "models/deepfake_detector_model.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
saliency_partitioned_save_path = "freq_part_imgs/"

# Training ResNet-18 Model

In [ ]:
# load data
train_subset = util.load_data.get_subset('Train', data_percentage=PERCENT_OF_DATA)
test_subset = util.load_data.get_subset('Test', data_percentage=PERCENT_OF_DATA)
validation_subset = util.load_data.get_subset('Validation', data_percentage=PERCENT_OF_DATA)

train_loader = util.load_data.load_dataset(train_subset, batch_size=32)
test_loader = util.load_data.load_dataset(test_subset, batch_size=32)
validation_loader = util.load_data.load_dataset(validation_subset, batch_size=32)

In [ ]:
model = util.dfnn_model.DeepfakeDetectorCNN(num_classes=num_classes)

# Example training setup
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
util.train_model.train_with_validation(model, train_loader, validation_loader, criterion, optimizer, device)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10] - Training:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [03:00<00:00,  3.28s/it]


Training Loss: 0.4030
Validation Loss: 0.3451, Validation Accuracy: 89.02%

Epoch [2/10] - Training:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [02:58<00:00,  3.25s/it]


...

Training Loss: 0.0582
Validation Loss: 0.3681, Validation Accuracy: 88.62%

Epoch [9/10] - Training:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [03:04<00:00,  3.36s/it]


Training Loss: 0.0400
Validation Loss: 0.6391, Validation Accuracy: 84.76%

Epoch [10/10] - Training:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [03:06<00:00,  3.39s/it]


Training Loss: 0.0622
Validation Loss: 0.3141, Validation Accuracy: 87.80%



In [11]:
test_loss, test_accuracy = util.train_model.test(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%\n")

Test Loss: 0.2628, Test Accuracy: 84.56%



In [12]:
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to models/deepfake_detector_model.pth


# Disjoint Ensembles Against Adversarial Examples

## Ensemble Model

In [7]:
ensemble_models = []
for i in range(NUM_ENSEMBLE_MODELS):
    model = util.dfnn_model.DeepfakeDetectorCNN(num_classes=2)
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)
    model.eval()  # Set to eval mode
    ensemble_models.append(model)

## Single Model

In [3]:
single_model = util.dfnn_model.DeepfakeDetectorCNN(num_classes=2)
single_model.load_state_dict(torch.load(model_save_path))
single_model.to(device)
single_model.eval()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DeepfakeDetectorCNN(
  (cnn): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

## Comparing

In [5]:
# create folder for subset of data
util.subset_data.create_subset_from_existing_dataset("Deepfake_Dataset/", "Subset_Dataset/", PERCENT_OF_DATA)

Subset dataset created at Subset_Dataset/


In [4]:
# Example: Load an image and model, then partition the image

image = util.saliency_partitioning.load_image('Subset_Dataset/Test/Fake/fake_79.jpg')  # Load an image as a numpy array

partitioned_images = util.saliency_partitioning.partition_image_for_ensemble(single_model, image)

AssertionError: Torch not compiled with CUDA enabled

In [3]:
# load data
test_noise_subset = util.load_data.get_subset('Test', data_percentage=PERCENT_OF_DATA, noise=True)
test_noise_loader = util.load_data.load_dataset(test_noise_subset, batch_size=32)

#creating the saliency partitioned images
util.ensemble.preprocess_images(test_noise_subset, save_dir=saliency_partitioned_save_path)

Processed 0/136 images.
Processed 100/136 images.


In [ ]:
plt.imshow(partitioned_image, cmap='gray')
plt.show()

In [ ]:


from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

partitioned_dataset = util.ensemble.SaliencyPartitionDataset(
    dataset=test_noise_loader.dataset,
    transform=transform,
    num_partitions=NUM_ENSEMBLE_MODELS,
)
partitioned_dataloader = DataLoader(partitioned_dataset, batch_size=32, shuffle=True)

In [8]:
for data in partitioned_dataloader:
    print(f"Data type: {type(data[0][0])}")  # Check partition type
    print(f"Label type: {type(data[1])}")  # Check label type
    break

TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [7]:
# Calculate metrics
test_accuracy, average_loss = util.ensemble.get_scores(ensemble_models, partitioned_dataloader, device)

# Print metrics
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(f"Average Loss: {average_loss:.4f}")

TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>